# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [2]:
# !ls /gdrive/MyDrive/
# !mkdir /gdrive/MyDrive/emotionImage
# %cp /content/data/rowImg/* /gdrive/MyDrive/emotionImage
!mkdir /content/data
!mkdir /content/data/rowVid
%cp  -r /gdrive/MyDrive/DataEngineering/* /content/data/rowVid



# Load and prepare dataset


## Load video 

In [3]:
%cd /content

/content


In [ ]:

!pip install opencv-contrib-python


Create dirctories

In [5]:

import shutil , os
import numpy as np
import cv2 as cv
import os
trial, count = 10, 0
def gen_img(skipFrame):
  if os.path.exists('data/rowImg'):
    shutil.rmtree('data/rowImg')
  if os.path.exists('data/processed'):
    shutil.rmtree('data/processed')
  dirs = [
      'data',
      'data/rowVid',
      'data/classification',
      'data/rowImg',
      'data/processed',
      'data/processed/train',
      'data/processed/valid',
      'data/processed/test',
      'data/processed/train/labels',
      'data/processed/valid/labels',
      'data/processed/test/labels',
      'data/processed/train/images',
      'data/processed/valid/images',
      'data/processed/test/images']

  [os.makedirs(p) if not os.path.exists(p) else ''  for p in dirs]


  for video in os.listdir('data/rowVid'):
    # if trial>count:
      if video.endswith('mp4') or video.endswith('avi'):
        cap = cv.VideoCapture('/content/data/rowVid/'+video)
        # capture.set(cv2.CAP_PROP_POS_FRAMES, 100)
        cap.set(cv2.CAP_PROP_POS_FRAMES, 100)


        c=1
        success,image = cap.read()
        count = 0

      while cap.isOpened():
        ret, frame = cap.read()

        if ret:
          # cv2.imwrite('frame{:d}.jpg'.format(count), frame)
          
          cv.imwrite(f'/content/data/rowImg/'+ str(count) + '-' + video[:-4] + '.png', frame)
          count += skipFrame # i.e. at 30 fps, this advances one second
          cap.set(cv2.CAP_PROP_POS_FRAMES, count)
        else:
          cap.release()
          break
        

In [6]:
%cp -r /gdrive/MyDrive/DataEngineering/* /content/data/rowVid

# Load classifier model

In [7]:
%mkdir model

In [ ]:
!wget -O model/emotionModel.hdf5 https://github.com/Furkan-Gulsen/face-classification/raw/main/models/emotionModel.hdf5
# https://github.com/Furkan-Gulsen/face-classification/blob/main/emotionRecognition.py implementatino code

In [9]:


from keras.models import load_model
import numpy as np
import cv2

In [10]:
emotionModelPath = 'model/emotionModel.hdf5'  # fer2013_mini_XCEPTION.110-0.65
emotionClassifier = load_model(emotionModelPath, compile=False)
emotionTargetSize = emotionClassifier.input_shape[1:3]

In [11]:
def frame_to_prid(x,y,h,w,img):
        try:

          grayFrame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

          grayFace = grayFrame[y:y + h, x:x + w]
          
          grayFace = cv2.resize(grayFace, (emotionTargetSize))
      
          grayFace = grayFace.astype('float32')
          grayFace = grayFace / 255.0
          grayFace = (grayFace - 0.5) * 2.0
          grayFace = np.expand_dims(grayFace, 0)
          grayFace = np.expand_dims(grayFace, -1)
          emotion_prediction = emotionClassifier.predict(grayFace)
          emotion_probability = np.max(emotion_prediction)
          # print(emotion_prediction)
          # print(emotion_probability)
          emotion_label_arg = np.argmax(emotion_prediction)
          return str(emotion_label_arg)
        except:
          pass
          

In [12]:
emotions = {
    0: {
        "emotion": "Angry",
        "color": (193, 69, 42)
    },
    1: {
        "emotion": "Disgust",
        "color": (164, 175, 49)
    },
    2: {
        "emotion": "Fear",
        "color": (40, 52, 155)
    },
    3: {
        "emotion": "Happy",
        "color": (23, 164, 28)
    },
    4: {
        "emotion": "Sad",
        "color": (164, 93, 23)
    },
    5: {
        "emotion": "Suprise",
        "color": (218, 229, 97)
    },
    6: {
        "emotion": "Neutral",
        "color": (108, 72, 200)
    }
}


# Extract Faces from rowImage using mtcnn

In [ ]:
!pip install pybboxes mtcnn


In [31]:
from mtcnn import MTCNN
import cv2
from pybboxes import BoundingBox
import os
import pybboxes as pbx

from tqdm import tqdm

In [69]:
# def generate_lable(rowImgDir,(train,val,test),(trainDir,valDir,testDir)):
import random ,math
import shutil

def generate_lable(rowImgDir, tvt, tvt_dir):
  shuffled_lst = [x for x in os.listdir(rowImgDir) if os.path.exists(rowImgDir+x) and (x.endswith('.png') or x.endswith('.jpg'))]
  random.shuffle(shuffled_lst)
  listLen = len(os.listdir(rowImgDir))
  train_list = shuffled_lst[0 : math.floor(tvt[0]*listLen)]
  valid_list = shuffled_lst[math.floor(tvt[0]*listLen) : math.floor(tvt[1]*listLen)+math.floor(tvt[0]*listLen)]
  test_list = shuffled_lst[math.floor(math.floor(tvt[1]*listLen)+math.floor(tvt[0]*listLen)) :-1 ]
  dataLists = [train_list, valid_list, test_list]

  for i ,lis in enumerate(tqdm(dataLists)):  

    for il, fn in enumerate(lis):      
      if os.path.exists(rowImgDir+fn) and (fn.endswith('.png') or fn.endswith('.jpg')):
        img = cv2.cvtColor(cv2.imread(rowImgDir+fn), cv2.COLOR_BGR2RGB)
        detector = MTCNN()

        if os.path.exists(tvt_dir[i]+fn[:-4]+'.txt'+'.txt'):
          os.remove(tvt_dir[i]+fn[:-4]+'.txt'+'.txt')
        for face in detector.detect_faces(img):
          try:
            
              a = face['box']
              coco_bbox = BoundingBox.from_coco(*a)
              coco_bbox.image_size = (img.shape[1], img.shape[0])
              yolo_bbox = coco_bbox.to_yolo()

              #get prid
              prid = frame_to_prid(a[0], a[1], a[2], a[3], img) 

              shutil.copyfile(rowImgDir+fn, tvt_dir[i]+'images/'+fn)
             
              # save yolo bbox
              f = open(tvt_dir[i]+'labels/' +fn[:-4]+'.txt', "a")
              f.write(f''+ prid +' '+ str(yolo_bbox.values[0]) + ' '+ str(yolo_bbox.values[1]) + ' '+ str(yolo_bbox.values[2]) + ' '+ str(yolo_bbox.values[3]) + '\n')
              f.close()
              # copy to all
          except  Exception as e:
            print(e)
            continue
          # save croped faces to new folder new file
          

In [ ]:
# def generate_lable(rowImgDir,(train,val,test),(trainDir,valDir,testDir)):
gen_img(600)
generate_lable('/content/data/rowImg/', (.4, .3, .3), ('data/processed/train/', 'data/processed/valid/', 'data/processed/test/'))


# Train Yolo

[link text](https://)## Extract frames(images from video)

/content/tomato-disease-ofise-16/data.yaml

In [18]:
!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image


Ultralytics YOLOv8.0.20 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.4/78.2 GB disk)


%cd 

!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=25 imgsz=800 plots=True

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data="/content/data.yaml" epochs=50 imgsz=1000 plots=True

In [ ]:
!yolo task=detect mode=predict model=/content/runs/detect/train7/weights/best.pt conf=0.25 source=/content/data/processed/test/images save=True

In [21]:
# for imgName in os.listdir('/content/runs/detect/predict')[0:5]:
# Image(filename='/content/runs/detect/predict6/2050-carpoolKaraoke.png', height=600)

In [22]:
# %cp -r /content/data/processed/test/images/* /gdrive/MyDrive/ml/emotionImg